# Comparação de Distâncias no Algoritmo KNN

Alunos: Vinícius de Sousa Paula e Lucas Campelo Santiago.

Este estudo tem como objetivo demonstrar a aplicação do algoritmo KNN (K-Nearest Neighbors) no processo de classificação de dados. Serão explorados dois métodos de cálculo de distância entre vetores: a distância euclidiana e a distância de Canberra. O propósito é analisar qual abordagem de distância resulta em melhor desempenho computacional e maior precisão na classificação.

In [2]:
# Instalação de dependências.
!pip install opencv-python numpy matplotlib scikit-learn

import os, random, sys, time
import numpy as np
import matplotlib.pyplot as plt
import cv2
from sklearn.model_selection import train_test_split

  Using cached opencv_python-4.9.0.80-cp37-abi3-manylinux_2_17_aarch64.manylinux2014_aarch64.whl.metadata (20 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 MB 20.0 MB/s eta 0:00:0000:0100:01


O código abaixo percorre um diretório e seus subdiretórios em busca de arquivos com extensão '.jpg'. Para cada arquivo encontrado, ele extrai a classe da imagem a partir do nome do diretório onde está localizado o arquivo e adiciona essa classe a uma lista enquanto armazena o caminho completo do arquivo em outra lista.

In [ ]:
directory = '../datasets/plf50'
images, classes = [], []

for root, dirs, files in os.walk(directory):
    for filename in files:
        if filename.endswith('.jpg'):
            file_path = os.path.join(root, filename)

            # Ler a classe da imagem retirando informação do nome do arquivo.
            splitted_string = file_path.split('/')
            image_class = splitted_string[-2]
            classes.append(image_class)

            images.append(file_path)

Funções fornecidas no exemplo.

In [ ]:
def save_img(path, img):
    files = path.split("\\")
    os.makedirs(files[0]+"2\\"+files[1], exist_ok=True) #cria as pastas onde a imagem cópia ficará
    cv2.imwrite(files[0]+"2\\"+files[1]+"\\"+files[2], 255*img, [cv2.IMWRITE_JPEG_QUALITY, 100]) #escreve a imagem no diretorio da
                                                                                                 #sua classe com qualidade jpeg 100
    return                                                                                       #e escala de 0 a 255

def read_img_to_gray_scale(path, size_x=16, size_y=16): #lê a imagem na escala cinza, a redimensiona e normaliza os valores para [0, 1]
    return cv2.resize(cv2.imread(path, 0)/255.0, (size_x, size_y))

def read_img_to_gray_scale_vector(path, size_x=16, size_y=16): #identico a readImgToGrayScale, mas transforma o resultado em vetor unidimensional
    return cv2.resize(cv2.imread(path, 0)/255.0, (size_x, size_y)).flatten()

def gray_scale_vector_to_img(vector, size_x=16, size_y=16): #retorna um vetor unidimensional para forma de matriz original (size_x, size_y)
    return np.reshape(vector, (size_x, size_y))

def plot_img(img): #visualiza a imagem
    plt.imshow(img, cmap="gray")
    plt.show()
    return

No código abaixo calculamos o número total de imagens e o número necessário de linhas de subplots para a grade de 10 imagens por linha. Em seguida, criamos uma figura e uma grade de subplots usando `plt.subplots()`. Iteramos sobre cada posição na grade de subplots e plotamos as imagens correspondentes. Se houver menos de 10 imagens em uma linha, os subplots vazios serão desativados.

In [ ]:
plot_images = []

for image in images:
    plot_images.append(plt.imread(image))
    
num_images = len(plot_images)
num_rows = (num_images + 9) // 10  # Arredonda para cima a divisão de num_images por 10

fig, axs = plt.subplots(num_rows, 10, figsize=(20, 5*num_rows))

for i in range(num_rows):
    for j in range(10):
        index = i * 10 + j
        if index < num_images:
            axs[i, j].imshow(plot_images[index])
            axs[i, j].axis('off')  # Desativa os eixos
            axs[i, j].set_title(f'Imagem {index + 1}')  # Define o título para cada imagem
        else:
            axs[i, j].axis('off')  # Desativa os eixos para subplots vazios

plt.tight_layout()  # Ajusta automaticamente o layout para evitar sobreposições
plt.show()

O código a seguir cria uma lista `x` contendo vetores unidimensionais representando as imagens após processamento para escala de cinza. Em seguida, as classes das imagens são armazenadas em uma lista `y`. Posteriormente, ele divide os dados em conjuntos de treinamento (`x_train`, `y_train`) e teste (`x_test`, `y_test`) usando a função `train_test_split` da biblioteca `scikit-learn`, com 30% dos dados destinados ao conjunto de teste.

In [ ]:
x = []

# Juntar todos os vetores unidimensionais das imagens na lista X.
for image in images:
    x.append(read_img_to_gray_scale_vector(image))

# As classes das imagens, mas em uma lista com nome Y.
y = classes

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42, shuffle=True)

A distância euclidiana entre dois vetores no mesmo espaço vetorial é definida por:

$$ D(\mathbf{p}, \mathbf{q}) = \sqrt{\sum_{i=1}^{n} (p_i - q_i)^2} $$


In [ ]:
# Função implementada seguindo a definição descrita em:
# https://en.wikipedia.org/wiki/Euclidean_distance
def euclidean_distance(p, q):
    return np.sqrt(np.sum((np.array(p) - np.array(q)) ** 2))

A distância de Canberra entre dois vetores no mesmo espaço vetorial é definida por:

$$ D = \sum_{i=1}^{n} \frac{|x_i - y_i|}{|x_i| + |y_i|} $$

In [ ]:
# Função implementada seguindo a definição fornecida pelo Vinícius:
def canberra_distance(x, y):
    denominator = np.abs(x) + np.abs(y)
    denominator[denominator == 0] = np.nan
    return np.nansum(np.abs(x - y) / denominator)

Definir função do algoritmo K-Nearest Neighbor (KNN) para K=1.

In [ ]:
# Função implementada seguindo a definição descrita em:
# https://www.ibm.com/topics/knn#:~:text=The%20k%2Dnearest%20neighbors%20(KNN)%20algorithm%20is%20a%20non,used%20in%20machine%20learning%20today.
def knn(x_train, y_train, train_data, distance_function):
    min_distance = float('inf')
    nearest_neighbor_class = None

    for index, x in enumerate(x_train):
        distance = distance_function(train_data, x)
        if distance < min_distance:
            min_distance = distance
            nearest_neighbor_class = y_train[index]

    return nearest_neighbor_class

A seguir serão executados os testes utilizando o algoritmo KNN com ambas as distâncias propostas (Euclidiana e Canberra).

Distância Euclidiana:

In [ ]:
# Iterar entre os elementos do conjunto de teste (x_test) e executar o algoritmo.
avg_time, accuracy = 0, 0

for index, x in enumerate(x_test):
    start = time.time() # Iniciar cronômetro.
    result = knn(x_train, y_train, x, euclidean_distance) # Realizar teste e extrair resultado.
    end = time.time() # Parar cronômetro.

    total_time = end - start # Medir tempo decorrido.
    avg_time += total_time # Armazenar tempo decorrido na variável de tempo médio.

    # Testar acurácia do teste.
    if result == y_test[index]:
        accuracy += 1

print('tempo médio de execução: ', round(avg_time / len(x_test), 5), 'segundos')
print('acurácia dos testes: ', round(accuracy / len(x_test), 2))

Distância de Canberra:

In [ ]:
# Iterar entre os elementos do conjunto de teste (x_test) e executar o algoritmo.
avg_time, accuracy = 0, 0

for index, x in enumerate(x_test):
    start = time.time() # Iniciar cronômetro.
    result = knn(x_train, y_train, x, canberra_distance) # Realizar teste e extrair resultado.
    end = time.time() # Parar cronômetro.

    total_time = end - start # Medir tempo decorrido.
    avg_time += total_time # Armazenar tempo decorrido na variável de tempo médio.

    # Testar acurácia do teste.
    if result == y_test[index]:
        accuracy += 1

print('tempo médio de execução: ', round(avg_time / len(x_test), 5), 'segundos')
print('acurácia dos testes: ', round(accuracy / len(x_test), 2))